# Processing

## Preparation

In [1]:
import rasterio
import queue
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
from collections import Counter, namedtuple
from src.utils import get_data_dir, benchmark_context


# Convenient access to data directory, is a namedtuple with folder names as attributes
DIRS = get_data_dir(str(Path('data').resolve()))

# Many functions of the processing pipeline are multi-threaded this attribute controls
# max number of threads
THREADLIMIT = 12

WGS84 = {'init': 'epsg:4326'}


def read_raster(item):
    """
    Helper method to return a raster file as a opened instance of
    rasterio.io.DatasetReader in read mode.
    
    :param item: str, pathlib.Path or rasterio.io.DatasetReader
        Should be the path to the raster file on filesystem as a string
        or pathlib.Path object. If item is a instance of DatasetReader
        the function returns immediately.
    :return: rasterio.io.DatasetReader
        Retruns an instance of rasterio.io.DatasetReader in read mode.
    """
    if isinstance(item, rasterio.io.DatasetReader):
        return item
    else:
        try:
            path = str(item)  # Cast pathlib.Path to string
            return rasterio.open(path, 'r')
        except:
            msg = 'Attr {}, Type {} is not a valid raster file'.format(item, type(item))
            raise ValueError(msg)


def binary_jaccard(arr1, arr2, return_matrix=False):
    """
    Calculates the Jaccard Index (JI) of two equal sized binary arrays or vectors.
    If return_matrix is set to true the method provides the JI and the necessary 
    calculation matrix as a named tuple. Attention, this method does not work in-place!
    
    :param arr1, arr2: numpy.ndarray, list, tuple
        Both array alike objects sized in equal dimensions should contain exclusively 
        binary data (1,0). 
    :param return_matrix: boolean
        Optional, a boolean value determining the return of the calculation matrix. 
    :return: float OR (float, namedtuple(m11, m01, m10, m00))
        Defaultly, the method returns only the JI if, return_matrix is set to true the 
        method returns the JI and the computation matrix.
        The Matrix contains the following attributes:
        m11 = total number of attributes where arr1 == 1 and arr2 == 1
        m10 = total number of attributes where arr1 == 1 and arr2 == 0
        m01 = total number of attributes where arr1 == 0 and arr2 == 1
        m00 = not required, set to 0
    """
    A, B = np.array(arr1, dtype=np.int8), np.array(arr2, dtype=np.int8)
    
    if np.sum(np.logical_or(A<0,A>1)) != 0 or np.sum(np.logical_or(B<0,B>1)) != 0:
        raise ValueError('Attributes should contain only binary values')
  
    C = A + B
    a = (B - C) + B  # a = (A - C) + A, m10 = a == 1
    b = (A - C) + A  # b = (B - C) + B, m01 = b == 1

    # Total number of attributes where A == 1 and B == 1
    m11 = np.sum(C==2)
    # Total number of attributes where A == 1 and B == 0
    m10 = np.sum(a==-1)
    # Total number of attributes where A == 0 and B == 1
    m01 = np.sum(b==-1)
    
    jaccard = m11 / (m10 + m01 + m11)
    
    if return_matrix:
        Matrix = namedtuple('Matrix', 'm11 m10 m01 m00')
        return jaccard, Matrix(m11, m10, m01, 0)
    return jaccard


def simple_matching_coefficient(arr1, arr2, return_matrix=False):
    """
    Calculates the Simple Matching Coefficient (SMC) of two equal sized arrays or vectors.
    If return_matrix is set to true the method provides the SMC and the necessary calculation 
    matrix as a named tuple. Attention, this method does not work in-place!
    
    :param arr1, arr2: numpy.ndarray, list, tuple
        Both array alike objects sized in equal dimensions should contain exclusively 
        binary data (1,0).
    :param return_matrix: boolean
        Optional, a boolean value determining the return of the calculation matrix.
    :return: float OR (float, namedtuple(m11, m01, m10, m00))
        Defaultly, the method returns only the SMC, if return_matrix is
        set to true the method returns the SMC and the computation matrix.
        The Matrix contains the following attributes:
        m11 = total number of attributes where arr1 == 1 and arr2 == 1
        m10 = total number of attributes where arr1 == 1 and arr2 == 0
        m01 = total number of attributes where arr1 == 0 and arr2 == 1
        m00 = total number of attributes where arr1 == 0 and arr2 == 0
    """
    _, matrix = binary_jaccard(arr1, arr2, True)
    A = np.array(arr1, dtype=np.int8)
    
    # Total number of attributes where A == 0 and B == 0
    m00 = A.size - sum(matrix)
    
    smc = (matrix.m11 + m00) / A.size

    if return_matrix:
        matrix = matrix._replace(m00=m00)
        return smc, matrix
    return smc

## Class harmonization
Workflow
- consider to use additional classes from gl30 wetlands or tundra
- initial
    - select forest (class value 20) from dataset gl30 - 2000
    - recode values to binary format 20 = 1, 0 = 0
    - select forest (class value 0 - 100) from hansen tree cover 2000
    - recode values to binary format 1 - 100 = 1, 0 = 0
    - calculate Jaccard Index with chen and hansen
- looping
    - select forest (0 + 10) - 100 from hansen tree cover 2000
    - recode values to binary format (0 + 10) - 100 = 1, 0 = 0
    - calculate Jaccard Index with chen and hansen
    - do till 30 or Jaccard Index is max
Potential Images
- world agreement map with different 
    - compare chen and hansen treccover in one image
    - sum of both dataset
    - 2 = agreement, 1 = disagreement

In [29]:
countries = gpd.read_file(str(DIRS.masks / 'ne_10m_admin_0_countries.shp'))
tiles = gpd.read_file(str(DIRS.masks / 'final_mask.shp'))
tiles.crs = WGS84

aio = countries.cx[:,-23:23]
aio = aio[['REGION_UN', 'geometry']]
continents = aio.dissolve(by='REGION_UN')

layer = gpd.sjoin(tiles, continents, how='left', op='intersects')
layer.to_file(str(DIRS.masks / 'test.shp'))

In [2]:
# Get rid of irelevant values is most convenient with
# arr[np.isin(arr, [1,3,4])] = 0

def worker(gl30, cover, queue, *args):
    r1 = read_raster(gl30)
    r2 = read_raster(cover)
    
    d1 = r1.read(1)
    d2 = r2.read(1)
    
    r1.close()
    r2.close()
    
    d1[d1!=20] = 0
    d1[d1==20] = 1
    
    tmp = d2.copy()
    
    args = list(args)
    for i in [0, 10, 20, 30]:
        tmp[d2<=i] = 0
        tmp[d2>i] = 1

        args.append(binary_jaccard(d1, tmp))
        args.append(simple_matching_coefficient(d1, tmp))
    
    queue.put(args)


# Tile region jc0 smc0 jc10 smc10 jc20 smc20 
mask = gpd.read_file(str(DIRS.masks / 'test.shp'))
idx = 0
q = queue.Queue()
for gl30, cover, region, key in zip(mask['gl30_00'], mask['cover'], mask['index_righ'], mask['key']):
    worker(DIRS.proc / gl30, DIRS.proc / cover, q, key, region)
    print(idx)
    idx += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in long_scalars


49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289


In [3]:
vals = []
while not q.empty():
    vals.append(q.get())

In [6]:
df = pd.DataFrame(vals)

In [8]:
df.columns = ['tile', 'region', 'jc0', 'smc0', 'jc10', 'smc10', 'jc20', 'smc20', 'jc30', 'smc30']

In [10]:
df.to_csv(str(DIRS.tmp / 'test.csv'))